<a href="https://colab.research.google.com/github/eteitelbaum/code-satp/blob/Fall-2024/training_damage_details_extraction_t5base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
!pip install datasets evaluate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=005bd47af09bbfd79800fc23e66574289768392197fb8697786170edb7115535
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load your dataset
data = pd.read_csv('/content/drive/MyDrive/SATP_data/property_damage.csv', dtype=str)

# Filter data to include cases with and without property damage
data_with_damage = data[data['value_property_damage'].notna()]
data_no_damage = data[data['value_property_damage'].isna()]

num_damage = len(data_with_damage)
print(f"Number of cases with property damage: {num_damage}")


Number of cases with property damage: 1366


In [ ]:

# Sample the same number of cases without property damage
data_no_damage_sampled = data_no_damage.sample(n=num_damage, random_state=42)

# Combine both datasets
balanced_data = pd.concat([data_with_damage, data_no_damage_sampled], ignore_index=True)
print(len(balanced_data))
# Replace NA values in 'value_property_damage' with 'NA'
balanced_data['value_property_damage'] = balanced_data['value_property_damage'].fillna('NA')

def prepare_data(df):
    inputs = [f"Extract the property damage value from the incident: {summary}" for summary in df['incident_summary']]
    targets = df['value_property_damage'].tolist()
    return {'input_text': inputs, 'target_text': targets}

dataset = Dataset.from_dict(prepare_data(balanced_data))


2732


In [ ]:

# Split into train, validation, and test sets (80% train, 10% val, 10% test)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
test_val_split = dataset['test'].train_test_split(test_size=0.5, seed=42)

# Merge splits into a DatasetDict
dataset = DatasetDict({
    'train': dataset['train'],
    'validation': test_val_split['train'],
    'test': test_val_split['test']
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 2185
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 273
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 274
    })
})


In [ ]:

import torch
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:

from transformers import T5Tokenizer

# Initialize the tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = examples['input_text']
    targets = examples['target_text']

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(targets, max_length=64, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply tokenization
tokenized_datasets = dataset.map(preprocess_function, batched=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/2185 [00:00<?, ? examples/s]

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

Map:   0%|          | 0/274 [00:00<?, ? examples/s]

In [ ]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate exact match
    exact_matches = [
        int(pred.strip().lower() == label.strip().lower()) for pred, label in zip(decoded_preds, decoded_labels)
    ]
    accuracy = sum(exact_matches) / len(exact_matches)

    return {"exact_match": accuracy * 100}  # Return as percentage


In [ ]:

from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory for checkpoints and logs
    evaluation_strategy="epoch",     # Evaluate at the end of every epoch
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Weight decay for regularization
    save_strategy="epoch",           # Save the model at each epoch
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=100,               # Log every 100 steps
    load_best_model_at_end=True,     # Load the best model at the end of training
    report_to='none'
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    #compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-749c3de5f54c>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

# Train the model
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.125200,0.072440
2,0.082100,0.065243
3,0.065400,0.063588


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=822, training_loss=0.312862749517399, metrics={'train_runtime': 1001.5259, 'train_samples_per_second': 6.545, 'train_steps_per_second': 0.821, 'total_flos': 3991718972620800.0, 'train_loss': 0.312862749517399, 'epoch': 3.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/MyDrive/SATP_data/damage_details_extraction/t5base_finetuned_model')
tokenizer.save_pretrained('/content/drive/MyDrive/SATP_data/damage_details_extraction/t5base_finetuned_model')


('/content/drive/MyDrive/SATP_data/damage_details_extraction/t5base_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/SATP_data/damage_details_extraction/t5base_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/SATP_data/damage_details_extraction/t5base_finetuned_model/spiece.model',
 '/content/drive/MyDrive/SATP_data/damage_details_extraction/t5base_finetuned_model/added_tokens.json')

In [ ]:
# Evaluate the model on the test set
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print(f"Test Set Evaluation Results: {eval_results}")


Test Set Evaluation Results: {'eval_loss': 0.07105866074562073, 'eval_runtime': 13.1461, 'eval_samples_per_second': 20.843, 'eval_steps_per_second': 5.249, 'epoch': 3.0}


In [ ]:
def predict_damage(summary):
    # Prepare the input text
    input_text = f"Extract the property damage value from the incident: {summary}"
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).input_ids

    # Generate predictions
    outputs = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)

    # Decode the output
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return predicted_text


Predicted damage: a porcelain and a tractor


In [ ]:

model.to('cpu')
# Example usage
incident_summary = 'Armed Communist Party of India-Maoist (CPI-Maoist) cadres as many as five tipper trucks on Keredari-Tandawa main road in Hazaribagh District of Jharkhand on September 30, reports organiser.org. Police said around 8-10 Maoists, armed with weapons, barged at the scene at around 2.30 am and set ablaze five Hyva vehicles parked at the roadside for coal transportation purposes. The extremists, after torching the vehicles, thrashed the drivers, threatening them to stop transportation. They also fired shots in the air before fleeing the spot. It’s been suspected that the extremists carried out the torching incident for a ‘levy’ (extortion money) amount; however, no confirmation about the same has been made so far.'
predicted_damage = predict_damage(incident_summary)
print(f"Predicted damage: {predicted_damage}")


Predicted damage: five Hyva vehicles set ablaze


In [ ]:

# Split into train, validation, and test sets (80% train, 10% val, 10% test)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
test_val_split = dataset['test'].train_test_split(test_size=0.5, seed=42)

# Merge splits into a DatasetDict
dataset = DatasetDict({
    'train': dataset['train'],
    'validation': test_val_split['train'],
    'test': test_val_split['test']
})

def preprocess_function(examples):
    inputs = examples['input_text']
    targets = examples['target_text']

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(targets, max_length=64, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


# Apply tokenization
tokenized_dataset_test = dataset["test"].map(preprocess_function, batched=True)
# Set format for PyTorch
tokenized_dataset_test.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'labels']
)


Map:   0%|          | 0/274 [00:00<?, ? examples/s]

In [ ]:
import numpy as np

def compute_metrics(predictions, labels):
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = [
        tokenizer.decode([l for l in label if l != -100], skip_special_tokens=True)
        for label in labels
    ]

    # Calculate exact match
    exact_matches = [
        int(pred.strip() == label.strip()) for pred, label in zip(decoded_preds, decoded_labels)
    ]
    accuracy = sum(exact_matches) / len(exact_matches)

    return {"exact_match": accuracy * 100}  # Return percentage


In [ ]:
import torch
from torch.utils.data import DataLoader

def evaluate_model(model, dataloader, tokenizer):
    model.eval()  # Set the model to evaluation mode
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation
        for batch in dataloader:
            # Move input data to the appropriate device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]

            # Generate predictions
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=64
            )
            predictions = outputs.cpu().numpy()  # Move predictions to CPU
            all_predictions.extend(predictions)
            all_labels.extend(labels.numpy())  # Move labels to CPU

    # Compute metrics
    metrics = compute_metrics(all_predictions, all_labels)
    return metrics


In [ ]:
from torch.utils.data import DataLoader

# Convert the test dataset to a PyTorch DataLoader
test_loader = DataLoader(tokenized_dataset_test, batch_size=8)


In [ ]:
# Evaluate the model on the test set
metrics = evaluate_model(model, test_loader, tokenizer)
print(f"Test Set Metrics: {metrics}")


Test Set Metrics: {'exact_match': 55.839416058394164}


In [ ]:
!pip install evaluate nltk rouge_score

In [ ]:
import evaluate

# Load BLEU and ROUGE evaluators
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(predictions, labels):
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = [
        tokenizer.decode([l for l in label if l != -100], skip_special_tokens=True)
        for label in labels
    ]

    # Format labels for BLEU (it expects a list of lists for references)
    formatted_labels = [[label] for label in decoded_labels]

    # Compute BLEU
    bleu = bleu_metric.compute(predictions=decoded_preds, references=formatted_labels)

    # Compute ROUGE
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Combine and return metrics
    metrics = {
        "bleu": bleu["bleu"] * 100,  # Convert to percentage
        "rouge1": rouge["rouge1"] * 100,
        "rouge2": rouge["rouge2"] * 100,
        "rougeL": rouge["rougeL"] * 100,
    }
    return metrics


In [ ]:
def evaluate_model(model, dataloader, tokenizer):
    model.eval()  # Set the model to evaluation mode
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation
        for batch in dataloader:
            # Move input data to the appropriate device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]

            # Generate predictions
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=64
            )
            predictions = outputs.cpu().numpy()  # Move predictions to CPU
            all_predictions.extend(predictions)
            all_labels.extend(labels.cpu().numpy())  # Move labels to CPU

    # Compute metrics
    metrics = compute_metrics(all_predictions, all_labels)
    return metrics


In [ ]:
# Evaluate the model on the test set
metrics = evaluate_model(model, test_loader, tokenizer)
print(f"Test Set Metrics: {metrics}")


Test Set Metrics: {'bleu': 32.66659426604134, 'rouge1': 77.28404469522047, 'rouge2': 22.602592154629626, 'rougeL': 76.26819416739224}


In [ ]:
def compute_metrics(predictions, labels):
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = [
        tokenizer.decode([l for l in label if l != -100], skip_special_tokens=True)
        for label in labels
    ]

    # Exact Match
    exact_matches = [
        int(pred.strip() == label.strip()) for pred, label in zip(decoded_preds, decoded_labels)
    ]
    exact_match_score = sum(exact_matches) / len(exact_matches)

    # Recall
    recalls = []
    for pred, label in zip(decoded_preds, decoded_labels):
        pred_components = set(pred.strip().split(", "))
        label_components = set(label.strip().split(", "))
        recall = len(pred_components & label_components) / len(label_components)
        recalls.append(recall)
    avg_recall = sum(recalls) / len(recalls)

    return {
        "exact_match": exact_match_score * 100,
        "recall": avg_recall * 100,
    }


In [ ]:
metrics = evaluate_model(model, test_loader, tokenizer)
print(f"Test Set Metrics: {metrics}")


Test Set Metrics: {'exact_match': 55.839416058394164, 'recall': 56.326034063260344}


In [ ]:

import pandas as pd

# Create an empty list to store the results
results = []

# Iterate over the test dataset
for i in range(len(tokenized_dataset_test)):
    input_data = tokenized_dataset_test[i]
    input_ids = input_data["input_ids"].unsqueeze(0).to(device)
    attention_mask = input_data["attention_mask"].unsqueeze(0).to(device)

    # Generate predictions
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=64)
    predicted_label = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Get the actual label
    actual_label = tokenizer.decode(input_data["labels"], skip_special_tokens=True)

    results.append({
        "predicted_label": predicted_label,
        "actual_label": actual_label
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv("test_predictions.csv", index=False)